1. 1 слой 26 нейронов (сначала передать туда матрицу декодера, потом инициализировать рандомно) 
2. 2 слоя по 31 нейрона
3. 

1. бинарный вход перевести в вещественный в [0,1]


In [1]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [2]:
# !mkdir -p drive
# !google-drive-ocamlfuse drive

In [3]:
import os 
import sys
import pandas as pd
import numpy as np

from keras import models
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Concatenate
# from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint


from sklearn import model_selection
from __future__ import absolute_import

# os.chdir("/content/drive/Hamming")
# sys.path.append("Hamming")
# import functions as f
# import model_lib 

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
!ls

best_model.hd5	functions.py	     model_lib.py  visualisation.ipynb
dataset_files	hamming-codes.ipynb  README.md


In [5]:
# !git pull

In [6]:
TRAIN_PATH = './dataset_files/hamming.txt'  # "/content/drive/Hamming/dataset_files/hamming.txt"

COLUMN_NAMES = ['id', 'plainword', 'codeword', 
                'id_error', 'bin_error', 'defective_codeword']

In [7]:
def hamming_distance(first: str, second: str) -> int:
    return len([1 for (x, y) in zip(first, second) if x != y])

In [8]:
def loss_hamming(y, y_pred):  # , w):
    return sum(1 for (a,b) in zip(y, y_pred) if y != y_pred) / y.shape[0] # надо ли делить????

In [9]:
def load_data():
    data = pd.read_csv(TRAIN_PATH, sep=';', names=COLUMN_NAMES)

    # make features
#     data['dec_defective_codeword'] = data['defective_codeword'][:].apply(lambda x: int(x, 2))
#     for j in range(len(data['defective_codeword'][0])):
#         data['bin_' + str(j)] = data['defective_codeword'][:].apply(lambda x: int(x[j]))
#     data['dec_plainword'] = data['plainword'][:].apply(lambda x: int(x, 2))
    return data

In [10]:
def make_features():
    data['dec_defective_codeword'] = data['defective_codeword'][:].apply(lambda x: int(x, 2))

    for j in range(len(data['codeword'][0])):
        data['cod_' + str(j)] = data['codeword'][:].apply(lambda x: int(x[j]))

    for j in range(len(data['defective_codeword'][0])):
        data['def_' + str(j)] = data['defective_codeword'][:].apply(lambda x: int(x[j]))

    for j in range(len(data['bin_error'][0])):
        data['mask_' + str(j)] = data['bin_error'][:].apply(lambda x: int(x[j]))
        
    for j in range(len(data['plainword'][0])):
        data['pln_' + str(j)] = data['plainword'][:].apply(lambda x: int(x[j]))


In [11]:
def split_data(test_size):
    train_data_defective, test_data_defective, train_labels_defective, test_labels_defective = \
    model_selection.train_test_split(data.loc[:, 'def_0':'def_30'], 
                                     data.loc[:, 'mask_0':'pln_25'], 
                                        test_size = test_size)
    
    train_data_plain, test_data_plain, train_labels_plain, test_labels_plain = \
    model_selection.train_test_split(data.loc[:, 'cod_0':'cod_30'], 
                                     data.loc[:, 'mask_0':'pln_25'], 
                                        test_size = test_size)
    
    train_data = np.vstack((np.array(train_data_defective), np.array(train_data_defective)))
    test_data = np.vstack((np.array(test_data_defective), np.array(test_data_defective)))
    train_labels = np.vstack((np.array(train_labels_defective), np.array(train_labels_defective)))
    test_labels = np.vstack((np.array(test_labels_defective), np.array(test_labels_defective)))
    
    return train_data, test_data, train_labels, test_labels

In [12]:
data = load_data() #pd.read_csv(functions.TRAIN_PATH, sep=';', names = functions.COLUMN_NAMES)

In [13]:
%%time
make_features()

CPU times: user 4.15 s, sys: 160 ms, total: 4.31 s
Wall time: 4.31 s


In [14]:
data.head()

,id,plainword,codeword,id_error,bin_error,defective_codeword,dec_defective_codeword,cod_0,cod_1,cod_2,...,pln_16,pln_17,pln_18,pln_19,pln_20,pln_21,pln_22,pln_23,pln_24,pln_25
0,0,10110000001111101100000100,0001001100000011111011000001000,8,0000000010000000000000000000000,0001001110000011111011000001000,163706376,0,0,0,...,1,1,0,0,0,0,0,1,0,0
1,1,00101011010000101000111100,1001101010110100001010001111000,17,0000000000000000010000000000000,1001101010110100011010001111000,1297757304,1,0,0,...,1,0,0,0,1,1,1,1,0,0
2,2,10010100110111111111111100,1101100101001101111111111111000,18,0000000000000000001000000000000,1101100101001101110111111111000,1822879736,1,1,0,...,1,1,1,1,1,1,1,1,0,0
3,3,00011000001101000100010111,1011100110000011010001000101110,13,0000000000000100000000000000000,1011100110000111010001000101110,1556324910,1,0,1,...,0,1,0,0,0,1,0,1,1,1
4,4,10011101101000001100111011,0011000111011010000011001110110,19,0000000000000000000100000000000,0011000111011010000111001110110,418188918,0,0,1,...,1,1,0,0,1,1,1,0,1,1


In [15]:
train_data, test_data, train_labels, test_labels = split_data(test_size=0.3)

In [16]:
decoder = []
with open('./dataset_files/h.txt') as f:
    decoder = pd.read_csv(f, header=None, names=['h'], dtype=str) #, sep='\n')
for j in range(len(decoder['h'][0])):
    decoder['bin_' + str(j)] = decoder['h'][:].apply(lambda x: int(x[j]))
decoder = decoder.drop(['h'], axis=1)
decoder_np = decoder.as_matrix()
print(decoder_np[:6])

[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [1 0 1 0 0]]


model = Sequential()
model.add(Dense(31, input_shape=(31,), 
#                 weights=decoder_np, 
                activation='relu', name='d1'))
# model.add(Dense(100, activation='relu', name='d2'))
# model.add(Dense(31, activation='softmax', name='d3'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy', # энтропия Шелдона / бинарная
              metrics=['accuracy', 'mse'])

MCP = ModelCheckpoint('best_model.hd5', save_best_only=True)

In [20]:
# train_labels_categor = to_categorical(train_labels, num_classes=31)

model.fit(x=train_data, y=train_labels_categor, validation_split=0.2,  shuffle=True, epochs=100, callbacks=[MCP])

LSTM;

матринчая нейронная сеть, каждый слой - логические ячейки xor...
метрика хэмминга

подавать и правильные и неправильные


поставить керас
from keras... import
inp = Input(shape=(31,))
x = Dense(128, activation='elu')(inp)
x = Dense(128, activation='relu')(x)

test = Dense(.....)(x)

m = Dense(26, activation='пороговая функция типа step которой уже нет 'sigmoid')(x)

test = Dense(...)(test)
test= Dense(31....)(test)
m = Concatinate()([test,m])
          
model = Model(inp, m, 'm')
model.compile() , summary, fit

метки = бинарный код ошибки + информационное слово

лосс - бинарная кросс энтропия
оптимизатор - Адам
стартовая точноесть - 1е-5

In [19]:
inp = Input(shape=(31,))
x = Dense(128, activation='relu',  name='d1')(inp)
x = Dense(128, activation='relu',  name='d2')(x)

t = Dense(128, activation='elu', name='t1')(x)
t = Dense(128, activation='relu', name='t2')(t)
t = Dense(31, activation='relu', name='t3')(t)

m = Dense(26, activation='sigmoid',  name='d3')(x)
m = Concatenate()([t, m])

model = Model(inp, m, 'm')
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='Adam')
model.fit(x=train_data, y=train_labels, validation_split=0.2,  shuffle=True, epochs=100)#, callbacks=[MCP])

Train on 73400 samples, validate on 18350 samples
Epoch 1/100
73400/73400 [==============================] - 7s 94us/step - loss: 0.3898 - acc: 0.9442 - val_loss: 0.3617 - val_acc: 0.9593
Epoch 2/100
73400/73400 [==============================] - 7s 92us/step - loss: 0.3618 - acc: 0.9592 - val_loss: 0.3614 - val_acc: 0.9592
Epoch 3/100
73400/73400 [==============================] - 7s 100us/step - loss: 0.3616 - acc: 0.9591 - val_loss: 0.3612 - val_acc: 0.9591
Epoch 4/100
73400/73400 [==============================] - 7s 95us/step - loss: 0.3613 - acc: 0.9592 - val_loss: 0.3604 - val_acc: 0.9594
Epoch 5/100
73400/73400 [==============================] - 7s 97us/step - loss: 0.3611 - acc: 0.9592 - val_loss: 0.3613 - val_acc: 0.9591
Epoch 6/100
73400/73400 [==============================] - 7s 91us/step - loss: 0.3608 - acc: 0.9592 - val_loss: 0.3611 - val_acc: 0.9594
Epoch 7/100
73400/73400 [==============================] - 7s 89us/step - loss: 0.3606 - acc: 0.9592 - val_loss: 0.3609 -

73400/73400 [==============================] - 7s 101us/step - loss: 0.3560 - acc: 0.9597 - val_loss: 0.3593 - val_acc: 0.9596
Epoch 60/100
73400/73400 [==============================] - 7s 96us/step - loss: 0.3559 - acc: 0.9597 - val_loss: 0.3591 - val_acc: 0.9596
Epoch 61/100
73400/73400 [==============================] - 7s 91us/step - loss: 0.3560 - acc: 0.9597 - val_loss: 0.3588 - val_acc: 0.9597
Epoch 62/100
73400/73400 [==============================] - 7s 95us/step - loss: 0.3559 - acc: 0.9597 - val_loss: 0.3590 - val_acc: 0.9596
Epoch 63/100
73400/73400 [==============================] - 7s 96us/step - loss: 0.3559 - acc: 0.9597 - val_loss: 0.3590 - val_acc: 0.9596
Epoch 64/100
73400/73400 [==============================] - 6s 88us/step - loss: 0.3558 - acc: 0.9597 - val_loss: 0.3586 - val_acc: 0.9596
Epoch 65/100
73400/73400 [==============================] - 7s 91us/step - loss: 0.3558 - acc: 0.9598 - val_loss: 0.3589 - val_acc: 0.9597
Epoch 66/100
73400/73400 [=============